<a href="https://colab.research.google.com/github/harikrishna-chem/opv_aem_2018/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook imports models published in the article titled "Toward predicting efficiency of organic solar cells via machine learning and improved descriptors" and offers predictions for the power conversion efficiency (PCE) of new candidates. For more detailed information, please refer to
[Adv. Energy Mater. 2018, 8, 1801032](https://doi.org/10.1002/aenm.201801032).

#### Install required dependencies

In [27]:
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

!./bin/micromamba create -y -n opv_legacy -c conda-forge \
  python=3.11 scikit-learn=1.2.2 numpy=1.26.4 scipy pandas joblib threadpoolctl

bin/micromamba
[+] 0.0s
[+] 0.1s
conda-forge/linux-64   1%
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   9%
conda-forge/noarch    14%[+] 0.3s
conda-forge/linux-64  18%
conda-forge/noarch    30%[+] 0.4s
conda-forge/linux-64  23%
conda-forge/noarch    42%[+] 0.5s
conda-forge/linux-64  29%
conda-forge/noarch    54%[+] 0.6s
conda-forge/linux-64  31%
conda-forge/noarch    58%[+] 0.7s
conda-forge/linux-64  35%
conda-forge/noarch    67%[+] 0.8s
conda-forge/linux-64  35%
conda-forge/noarch    67%[+] 0.9s
conda-forge/linux-64  35%
conda-forge/noarch    67%[+] 1.0s
conda-forge/linux-64  35%
conda-forge/noarch    67%[+] 1.1s
conda-forge/linux-64  35%
conda-forge/noarch    67%[+] 1.2s
conda-forge/linux-64  35%
conda-forge/noarch    67%[+] 1.3s
conda-forge/linux-64  37%
conda-forge/noarch    67%[+] 1.4s
conda-forge/linux-64  37%
conda-forge/noarch    67%[+] 1.5s
conda-forge/linux-64  37%
conda-forge/noarch    67%[+] 1.6s
conda-forge/linux-64  37%
conda-forge/noarch    67%[+] 1.7s
conda-f

In [28]:
!./bin/micromamba run -n opv_legacy python -c "import sys, sklearn; print(sys.version); print('sklearn', sklearn.__version__)"

3.11.14 | packaged by conda-forge | (main, Oct 22 2025, 22:46:25) [GCC 14.3.0]
sklearn 1.2.2


In [29]:
%%bash
./bin/micromamba run -n opv_legacy python - <<'PY'
import sys, sklearn, pickle
import sklearn.tree._tree as _tree

print("python:", sys.version)
print("sklearn:", sklearn.__version__)
print("_tree:", _tree.__file__)

with open('/usr/local/opv_aem_2018/models/rf_aem.pkl', 'rb') as f:
    rf = pickle.load(f)

with open('/usr/local/opv_aem_2018/models/gbrt_aem.pkl', 'rb') as f:
    gbrt = pickle.load(f)

print("Loaded RF:", type(rf))
PY

python: 3.11.14 | packaged by conda-forge | (main, Oct 22 2025, 22:46:25) [GCC 14.3.0]
sklearn: 1.2.2
_tree: /root/.local/share/mamba/envs/opv_legacy/lib/python3.11/site-packages/sklearn/tree/_tree.cpython-311-x86_64-linux-gnu.so
Loaded RF: <class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [30]:
if not hasattr(dm, "EuclideanDistance") and hasattr(dm, "EuclideanDistance64"):
    dm.EuclideanDistance = dm.EuclideanDistance64  # alias

#### Predict the PCE for a Donor-Acceptor Pair

In [31]:
#@title Predict the PCE for a Donor-Acceptor Pair
#@markdown ***Please first input your parameters, and then press on the left button to run.***

# Import packages
import os
import pickle
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Import models
if os.path.exists("/usr/local/opv_aem_2018"):
  pass
else:
  %cd /usr/local
  !git clone -q https://github.com/harikrishna-chem/opv_aem_2018.git

# Load models
# Load the LR model
with open('/usr/local/opv_aem_2018/models/lr_aem.pkl', 'rb') as file:
    lr = pickle.load(file)

# Load the ANN model
with open('/usr/local/opv_aem_2018/models/ann_aem.pkl', 'rb') as file:
    ann = pickle.load(file)

# Load the KNN model
with open('/usr/local/opv_aem_2018/models/knn_aem.pkl', 'rb') as file:
    knn = pickle.load(file)

# Load the RF model
#with open('/usr/local/opv_aem_2018/models/rf_aem.pkl', 'rb') as file:
#    rf = pickle.load(file)

# Load the GBRT model
#with open('/usr/local/opv_aem_2018/models/gbrt_aem.pkl', 'rb') as file:
#    gbrt = pickle.load(file)

# @markdown ** This will predict the PCE for a new material using given molecular properties calculated at the M06-2X/6-31G(d) level.:**

# @markdown 1. Polarizability of the donor molecule (Bohr$^3$):
polarizability = 1143.852667 # @param {type:"number"}
# @markdown 2. The energetic difference of LUMO and LUMO+1 of the acceptor, $\Delta_{\rm L}^{\rm A}$ (eV):
delLA = 0.033470 # @param {type:"number"}
# @markdown 3. The energetic difference of LUMO and LUMO+1 of the donor, $\Delta_{\rm L}$ (eV):
delLD = 0.047620 # @param {type:"number"}
# @markdown 4. Number of unsaturated atoms in the main conjugation path of the donor molecule, $N_{\rm atom}^{\rm D}$ :
N_atom = 58 # @param {type:"number"}
# @markdown 5. Energy of the electronic transition to a singlet excited state with the largest oscillator strength, $E_{\rm g}$ (nm):
Eg = 453.34 # @param {type:"number"}
# @markdown 6. Reorganization energy for holes in the donor molecule, $\lambda_{\rm h}$ (eV):
lambda_h = 0.384475 # @param {type:"number"}
# @markdown 7. Vertical ionization potential for the donor molecule, IP($v$) (eV):
DIP = 6.522940 # @param {type:"number"}
# @markdown 8. The energetic difference of HOMO of donor and LUMO of acceptor, $E_{\rm HL}^{\rm DA}$ (eV):
AL_DH = 3.650407 # @param {type:"number"}
# @markdown 9. The energetic differences of HOMO and HOMO−1 for the donor molecule, $\Delta_{\rm H}$ (eV):
delHD = 0.345585 # @param {type:"number"}
# @markdown 10. Hole–electron binding energy in the donor molecule, $E_{\rm bind}$ (eV):
E_bind = 2.080714 # @param {type:"number"}
# @markdown 11. The energetic difference of LUMO of donor and LUMO of acceptor, $E_{\rm LL}^{\rm DA}$ (eV):
DL_AL = 0.500962 # @param {type:"number"}
# @markdown 12. Change in dipole moment in going from the ground state to the first excited state for donor molecule, $\Delta_{\rm ge}$ (Debye):
delGE = 1.282700 # @param {type:"number"}
# @markdown 13. Energy of the electronic transition to the lowest-lying triplet state, E$_{\rm T_1}$ (eV):
E_T1 = 1.8899 # @param {type:"number"}

# Create a pandas DataFrame
column_names = ['polarizability', 'delLA', 'delLD', 'N_atom', 'Eg',
       'lamda_h', 'DIP', 'AL-DH', 'delHD', 'E_bind', 'DL-AL', 'delGE', 'E_T1']
input_data_list = [polarizability,delLA, delLD, N_atom, Eg, lambda_h, DIP, AL_DH, delHD,E_bind, DL_AL, delGE, E_T1]
input_data = pd.DataFrame(columns=column_names)
input_data.loc[len(input_data)] = input_data_list

### Descriptor scaling ###
train = pd.read_csv("/usr/local/opv_aem_2018/datasets/Train.csv")
trainX = train.to_numpy()[:,2:]
testX = input_data.to_numpy()
scalerS=StandardScaler()
scalerS.fit(trainX)
trainX_S = scalerS.transform(trainX)
testX_S = scalerS.transform(testX)

scalerM=MinMaxScaler()
scalerM.fit(trainX)
trainX_M = scalerM.transform(trainX)
testX_M = scalerM.transform(testX)

# Predict the PCE
list_pred_PCE = []

#LR
test_pred_LR = lr.predict(testX)
list_pred_PCE.extend(test_pred_LR)

#KNN
test_pred_KNN = knn.predict(testX_M)
list_pred_PCE.extend(test_pred_KNN)

#ANN
test_pred_ANN = ann.predict(testX_S)
list_pred_PCE.extend(test_pred_ANN)

#RF
test_pred_RF = rf.predict(testX)
list_pred_PCE.extend(test_pred_RF)

#GBRT
test_pred_GB = gbrt.predict(testX)
list_pred_PCE.extend(test_pred_GB)

model_list = ["LR", "kNN", "ANN", "RF", "GBRT"]
predicted_data = pd.DataFrame(columns=model_list)
predicted_data.loc[len(predicted_data)] = list_pred_PCE
predicted_data.index = ['D/A']

print("Predicted PCEs (%) using linear regression (LR), k-nearest neighbor (kNN), artificial neural networks (ANN), random forest (RF), and gradient boosting regression tree (GBRT) models are provided below:")
print(predicted_data)

Predicted PCEs (%) using linear regression (LR), k-nearest neighbor (kNN), artificial neural networks (ANN), random forest (RF), and gradient boosting regression tree (GBRT) models are provided below:
           LR       kNN       ANN        RF      GBRT
D/A  5.957946  6.894793  6.657736  5.957946  5.957946


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsRegressor from version 1.2.2 when using version 1.6.1. This might lead to breaking cod

In [ ]:
#@title Batch Prediction for Multiple Donor-Acceptor Pairs
#@markdown In this section, we will demonstrate how to utilize our models for predicting the Power Conversion Efficiency (PCE) of a number of donor-acceptor (D/A) pairs. Please follow the instructions provided on page 33 of the Supporting Information document from the AEM paper to compile a dataset in CSV format ("candidate.csv"), containing all required information for the donor-acceptor pairs you intend to use for PCE prediction. Once you have prepared your CSV file, upload it to Colab using the icon in the left column and then click the left button to initiate the process. To illustrate this, We will use a sample candidate dataset. Users can subsequently download the predicted results, which are saved in the 'predicted_PCE.csv' file accessible through the icon in the left column.
# Import packages
import os
import pickle
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Import models
if os.path.exists("/usr/local/opv_aem_2018"):
  pass
else:
  %cd /usr/local
  !git clone -q https://github.com/harikrishna-chem/opv_aem_2018.git

# Load models
# Load the LR model
with open('/usr/local/opv_aem_2018/models/lr_aem.pkl', 'rb') as file:
    lr = pickle.load(file)

# Load the ANN model
with open('/usr/local/opv_aem_2018/models/ann_aem.pkl', 'rb') as file:
    ann = pickle.load(file)

# Load the KNN model
with open('/usr/local/opv_aem_2018/models/knn_aem.pkl', 'rb') as file:
    knn = pickle.load(file)

# Load the KNN model
with open('/usr/local/opv_aem_2018/models/rf_aem.pkl', 'rb') as file:
    rf = pickle.load(file)

# Load the KNN model
with open('/usr/local/opv_aem_2018/models/gbrt_aem.pkl', 'rb') as file:
    gbrt = pickle.load(file)

if os.path.exists("/content/candidate.csv"):
    pass
else:
    !cp /usr/local/opv_aem_2018/datasets/Test.csv /content/candidate.csv

### Read Train and Test data ###
train = pd.read_csv("/usr/local/opv_aem_2018/datasets/Train.csv")
test = pd.read_csv("/content/candidate.csv")
trainX = train.to_numpy()[:,2:]
testX = test.to_numpy()[:,2:]

### Descriptor scaling ###
scalerS=StandardScaler()
scalerS.fit(trainX)
trainX_S = scalerS.transform(trainX)
testX_S = scalerS.transform(testX)

scalerM=MinMaxScaler()
scalerM.fit(trainX)
trainX_M = scalerM.transform(trainX)
testX_M = scalerM.transform(testX)

# Predict the PCE
#LR
test_pred_LR = lr.predict(testX)

#KNN
test_pred_KNN = knn.predict(testX_M)

#ANN
test_pred_ANN = ann.predict(testX_S)

#RF
test_pred_RF = rf.predict(testX)

#GBRT
test_pred_GB = gbrt.predict(testX)

# Create a pandas DataFrame for predicted PCEs
pred_df = pd.DataFrame() #test[:,0]
pred_df['SN'] = test.iloc[:, 0]
pred_df['LR'] = test_pred_LR
pred_df['kNN'] = test_pred_KNN
pred_df['ANN'] = test_pred_ANN
pred_df['RF'] = test_pred_RF
pred_df['GBRT'] = test_pred_GB

# Save the data in a CSV file
pred_df.to_csv("/content/predicted_PCE.csv")

print("Predicted PCEs (%) using linear regression (LR), k-nearest neighbor (kNN), artificial neural networks (ANN), random forest (RF), and gradient boosting regression tree (GBRT) models are provided below:")
print(pred_df)

Predicted PCEs (%) using linear regression (LR), k-nearest neighbor (kNN), artificial neural networks (ANN), random forest (RF), and gradient boosting regression tree (GBRT) models are provided below:
     SN        LR       kNN       ANN        RF      GBRT
0    49  5.957947  6.894791  6.657738  6.863784  7.537297
1   200  3.713405  4.709338  3.869178  3.341831  3.368673
2   267  1.403432  1.178437  1.898073  1.822651  1.799663
3    16  7.295168  8.020000  7.735599  7.583088  7.951715
4     4  6.436520  7.676667  7.237478  7.663054  7.715267
5    27  7.741166  7.821983  8.010157  6.994241  7.106885
6    77  5.554730  6.453740  5.572032  6.246980  6.051890
7   158  5.785386  6.013816  5.933792  5.197361  5.102075
8   147  6.386511  7.058471  6.849342  5.827725  6.054330
9   120  5.410510  4.537978  4.486281  5.318025  4.668129
10  105  4.364924  4.329751  4.228804  4.530946  4.582008
11  108  4.250078  4.956230  4.717048  5.109583  5.218335
12  181  6.366342  8.304319  7.057200  6.4827

# References and further reading

- Harikrishna Sahu,  Weining Rao,  Alessandro Troisi,  Haibo Ma, *Toward predicting efficiency of organic solar cells via machine learning and improved descriptors*, Adv. Energy Mater. 2018, 8, 1801032. DOI: [10.1002/aenm.201801032](https://doi.org/10.1002/aenm.201801032)